In [124]:
import numpy as np
import pandas as pd

In [148]:
links = pd.read_csv('data/links.csv')
movies = pd.read_csv('data/movies.csv')
ratings = pd.read_csv('data/ratings.csv')
tags = pd.read_csv('data/tags.csv')

In [149]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [150]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [151]:
tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


## movies data 전처리

In [152]:
duplicates = movies[movies.duplicated('title', keep=False)]
duplicates

,movieId,title,genres
650,838,Emma (1996),Comedy|Drama|Romance
2141,2851,Saturn 3 (1980),Adventure|Sci-Fi|Thriller
4169,6003,Confessions of a Dangerous Mind (2002),Comedy|Crime|Drama|Thriller
5601,26958,Emma (1996),Romance
5854,32600,Eros (2004),Drama
5931,34048,War of the Worlds (2005),Action|Adventure|Sci-Fi|Thriller
6932,64997,War of the Worlds (2005),Action|Sci-Fi
9106,144606,Confessions of a Dangerous Mind (2002),Comedy|Crime|Drama|Romance|Thriller
9135,147002,Eros (2004),Drama|Romance
9468,168358,Saturn 3 (1980),Sci-Fi|Thriller


In [153]:
movies['title'].value_counts()[movies['title'].value_counts() > 1]

Emma (1996)                               2
War of the Worlds (2005)                  2
Confessions of a Dangerous Mind (2002)    2
Eros (2004)                               2
Saturn 3 (1980)                           2
Name: title, dtype: int64

In [154]:
movies[movies['title'] == 'Emma (1996)']
movies = movies[~(movies['movieId'] == 26958)].copy()
ratings.loc[80596, 'movieId'] = 838
movies[movies['title'] == 'War of the Worlds (2005)']
movies = movies[~(movies['movieId'] == 64997)].copy()
ratings.loc[4747, 'movieId'] = 34048
ratings.loc[11451, 'movieId'] = 34048
movies[movies['title'] == 'Confessions of a Dangerous Mind (2002)']
movies.loc[4169, 'genres'] ='Comedy|Crime|Drama|Romance|Thriller'
movies = movies[~(movies['movieId'] == 144606)].copy()
ratings.loc[17819, 'movieId'] = 6003
movies[movies['title'] == 'Eros (2004)']
movies.loc[5854,'genres'] = 'Drama|Romance'
movies = movies[~(movies['movieId'] == 147002)].copy()
ratings.loc[49832, 'movieId'] = 32600
movies[movies['title'] == 'Saturn 3 (1980)']
movies = movies[~(movies['movieId'] == 168358)].copy()
ratings.loc[81458, 'movieId'] = 2851

In [155]:
print('movieId unique:',movies['movieId'].nunique(),'movie title unique:',movies['title'].nunique())

movieId unique: 9737 movie title unique: 9737


In [156]:
movies = movies.reset_index(drop=True)
ratings = ratings.reset_index(drop=True)

In [157]:
movies_links = pd.merge(movies, links, on='movieId')

In [158]:
movies_links[~movies_links['title'].str.contains('\(')]

,movieId,title,genres,imdbId,tmdbId
6058,40697,Babylon 5,Sci-Fi,105946,NaN
9029,140956,Ready Player One,Action|Sci-Fi|Thriller,1677720,333339.0
9089,143410,Hyena Road,(no genres listed),4034452,316042.0
9134,147250,The Adventures of Sherlock Holmes and Doctor W...,(no genres listed),229922,127605.0
9175,149334,Nocturnal Animals,Drama|Thriller,4550098,340666.0
9255,156605,Paterson,(no genres listed),5247022,370755.0
9363,162414,Moonlight,Drama,4975722,376867.0
9444,167570,The OA,(no genres listed),4635282,432192.0
9509,171495,Cosmos,(no genres listed),81846,409926.0
9510,171631,Maria Bamford: Old Baby,(no genres listed),6264596,455601.0


In [159]:
idx_year_dict={6058:1994, 9029:2018, 9089:2015, 9134:1980,
                9175:2016, 9255:2016, 9363:2016, 9444:2016,
                9509:1980, 9510:2017, 9520:2017,9606:2011 }

In [160]:
tmp = movies_links.copy()

In [161]:
for idx, year in idx_year_dict.items():
    if '(' not in tmp.loc[idx, 'title']:
        tmp.loc[idx, 'title'] = tmp.loc[idx, 'title'] + f" ({year})"

    tmp.loc[[idx]]

In [162]:
tmp[~tmp['title'].str.contains('\(')]

,movieId,title,genres,imdbId,tmdbId


In [163]:
tmp.loc[9513, 'title'] = 'Death Note: Desu nôto (2006)'

In [164]:
movies_links = tmp

In [165]:
movies_links['year'] = movies_links['title'].str.extract(r'\((\d{4})\)', expand=False)
movies_links.head()

,movieId,title,genres,imdbId,tmdbId,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862.0,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,113497,8844.0,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,113228,15602.0,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,114885,31357.0,1995
4,5,Father of the Bride Part II (1995),Comedy,113041,11862.0,1995


In [166]:
duplicates = movies_links[movies_links.duplicated(subset='title', keep=False)]
duplicates

,movieId,title,genres,imdbId,tmdbId,year


In [167]:
#Error: invalid literal for int() with base 10: '2006–2007'
try:
    movies_links['year'] = movies_links['year'].astype(int)
except ValueError as e:
    print(f"Error: {e}")

In [168]:
movies_links.to_csv("data/movies_v1.csv",index=False)
ratings.to_csv("data/ratings_v1.csv",index=False)